In [9]:
# ==========================
# Global Setup
# ==========================
import os, random
import numpy as np
import pandas as pd
from pathlib import Path

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

PROJECT_DIR = Path("stock_project")
DATA_DIR = PROJECT_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

REPORT_DIR = PROJECT_DIR / "reports"
PRED_DIR = REPORT_DIR / "predictions"
TAB_DIR  = REPORT_DIR / "tables"
FIG_DIR  = REPORT_DIR / "figures"
MODEL_DIR = PROJECT_DIR / "models"

for d in [RAW_DIR, PROC_DIR, PRED_DIR, TAB_DIR, FIG_DIR, MODEL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("PROJECT_DIR:", PROJECT_DIR.resolve())
print("PROC_DIR:", PROC_DIR.resolve())
print("PRED_DIR:", PRED_DIR.resolve())
print("TAB_DIR :", TAB_DIR.resolve())
print("FIG_DIR :", FIG_DIR.resolve())

PROJECT_DIR: /notebooks/stock_project
PROC_DIR: /notebooks/stock_project/data/processed
PRED_DIR: /notebooks/stock_project/reports/predictions
TAB_DIR : /notebooks/stock_project/reports/tables
FIG_DIR : /notebooks/stock_project/reports/figures


In [10]:
# ==========================
# Load Base Panel
# ==========================
panel_path = PROC_DIR / "panel.parquet"
assert panel_path.exists(), f"Missing {panel_path}. Build your panel first."

panel = pd.read_parquet(panel_path).copy()
panel["Date"] = pd.to_datetime(panel["Date"])
panel["target_date"] = pd.to_datetime(panel["target_date"])

required = ["Date","target_date","Ticker","split","target_ret"]
missing = [c for c in required if c not in panel.columns]
assert not missing, f"panel missing columns: {missing}"

print("panel shape:", panel.shape)
print("tickers:", panel["Ticker"].nunique())
print("split counts:\n", panel["split"].value_counts(dropna=False))
print(panel[["Date","target_date","Ticker","target_ret","split"]].head())

panel shape: (19776, 23)
tickers: 8
split counts:
 split
train    7720
val      6048
test     6008
Name: count, dtype: int64
        Date target_date Ticker  target_ret  split
0 2016-03-02  2016-03-03   AAPL    0.007417  train
1 2016-03-03  2016-03-04   AAPL    0.014767  train
2 2016-03-04  2016-03-07   AAPL   -0.011129  train
3 2016-03-07  2016-03-08   AAPL   -0.008280  train
4 2016-03-08  2016-03-09   AAPL    0.000890  train


In [11]:
# ==========================
# Integrity & Leakage Checks
# ==========================
df = panel.copy()

# Basic NaN checks
print("NaNs in target_ret:", df["target_ret"].isna().sum())

# Check split ordering by target_date (train < val < test ideally)
df = df.sort_values(["Ticker", "target_date"])
split_minmax = df.groupby("split")["target_date"].agg(["min","max"]).sort_index()
print("target_date min/max by split:\n", split_minmax)

# Ensure no target_date earlier than Date
bad = (df["target_date"] < df["Date"]).sum()
print("Rows where target_date < Date:", bad)

# Ensure no overlap of target_date between splits (not always strictly true but should usually be)
train_max = df.loc[df["split"]=="train", "target_date"].max()
val_min   = df.loc[df["split"]=="val", "target_date"].min()
val_max   = df.loc[df["split"]=="val", "target_date"].max()
test_min  = df.loc[df["split"]=="test", "target_date"].min()

print("train_max:", train_max)
print("val_min  :", val_min, "val_max:", val_max)
print("test_min :", test_min)

NaNs in target_ret: 0
target_date min/max by split:
              min        max
split                      
test  2023-01-03 2025-12-30
train 2016-03-03 2019-12-31
val   2020-01-02 2022-12-30
Rows where target_date < Date: 0
train_max: 2019-12-31 00:00:00
val_min  : 2020-01-02 00:00:00 val_max: 2022-12-30 00:00:00
test_min : 2023-01-03 00:00:00


In [12]:
# ==========================
#  Metrics Utilities 
# ==========================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def directional_accuracy(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 0.0) -> float:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean((y_true > eps) == (y_pred > eps)))

def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)

    corr = np.nan
    if np.std(y_true) > 0 and np.std(y_pred) > 0:
        corr = float(np.corrcoef(y_true, y_pred)[0, 1])

    dir_acc = directional_accuracy(y_true, y_pred, eps=0.0)

    return {"MAE": float(mae), "RMSE": float(rmse), "Corr": corr, "DirectionalAcc": float(dir_acc)}

def oos_r2_vs_baseline_matched(pred_df: pd.DataFrame, baseline_pred_path: Path) -> dict:
    if not baseline_pred_path.exists():
        return {s: np.nan for s in ["train","val","test"]}

    base = pd.read_parquet(baseline_pred_path).copy()
    base["target_date"] = pd.to_datetime(base["target_date"])
    base = base.rename(columns={"y_pred":"y_pred_baseline"})[["Ticker","target_date","y_pred_baseline"]]

    merged = pred_df.merge(base, on=["Ticker","target_date"], how="left", validate="many_to_one")
    out = {}

    for split in ["train","val","test"]:
        d = merged[merged["split"] == split].dropna(subset=["y_pred_baseline"])
        if len(d) == 0:
            out[split] = np.nan
            continue
        mse_model = float(np.mean((d["y_true"] - d["y_pred"])**2))
        mse_base  = float(np.mean((d["y_true"] - d["y_pred_baseline"])**2))
        out[split] = np.nan if mse_base <= 0 else float(1.0 - mse_model/mse_base)
    return out

print("Loaded: regression_metrics, directional_accuracy, oos_r2_vs_baseline_matched")

Loaded: regression_metrics, directional_accuracy, oos_r2_vs_baseline_matched


In [13]:
# ==========================
#  Feature Candidates 
# ==========================
from pandas.api.types import is_numeric_dtype

NON_FEATURES = {"Date","target_date","split","Ticker","target_ret","has_garch"}

def numeric_feature_candidates(df: pd.DataFrame) -> list[str]:
    feats = []
    for c in df.columns:
        if c in NON_FEATURES:
            continue
        if is_numeric_dtype(df[c]):
            feats.append(c)
    return sorted(feats)

feature_candidates_base = numeric_feature_candidates(panel)
print("Number of numeric features:", len(feature_candidates_base))
print("Sample features:", feature_candidates_base[:20])

Number of numeric features: 18
Sample features: ['DFF_diff_lag1', 'DFF_lag1', 'DGS10_diff_lag1', 'DGS10_lag1', 'SP500_lag1', 'mkt_ret_lag1', 'ret', 'ret_lag1', 'ret_lag10', 'ret_lag2', 'ret_lag3', 'ret_lag5', 'ret_vol10', 'ret_vol20', 'ret_vol5', 'sp500_ret_lag1', 'vix_level_lag1', 'vix_ret_lag1']


In [14]:
# ============================================================
# Baseline Mean Models (FULL COPY/PASTE, RUN TOP->BOTTOM)
# ============================================================

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

df = panel.copy()
df["Date"] = pd.to_datetime(df["Date"])
df["target_date"] = pd.to_datetime(df["target_date"])

train_df = df[df["split"]=="train"].copy()
val_df   = df[df["split"]=="val"].copy()
test_df  = df[df["split"]=="test"].copy()

def _save_pred_df(pred_df: pd.DataFrame, model_name: str) -> Path:
    pred_df = pred_df.copy()
    pred_df["target_date"] = pd.to_datetime(pred_df["target_date"])
    path = PRED_DIR / f"{model_name}.parquet"
    pred_df.to_parquet(path, index=False)
    print(f"Saved predictions: {path} | rows={len(pred_df):,}")
    return path

def _metrics_from_pred(pred_df: pd.DataFrame, model_name: str) -> pd.DataFrame:
    rows = []
    for split in ["train","val","test"]:
        d = pred_df[pred_df["split"]==split]
        if len(d)==0:
            continue
        m = regression_metrics(d["y_true"].to_numpy(), d["y_pred"].to_numpy())
        rows.append({"model":model_name,"split":split,"n":int(len(d)),**m})
    return pd.DataFrame(rows)

def _attach_matched_oos_r2(metrics_df: pd.DataFrame, pred_df: pd.DataFrame) -> pd.DataFrame:
    baseline_path = PRED_DIR / "baseline_ticker_mean.parquet"
    oos = oos_r2_vs_baseline_matched(pred_df, baseline_path)
    metrics_df = metrics_df.copy()
    metrics_df["OOS_R2_vs_baseline"] = metrics_df["split"].map(oos)
    return metrics_df

def _basic_pred_frame(df_sub: pd.DataFrame, y_pred: np.ndarray, model_name: str) -> pd.DataFrame:
    out = df_sub[["Date","target_date","Ticker","split"]].copy()
    out["model"] = model_name
    out["y_true"] = df_sub["target_ret"].to_numpy()
    out["y_pred"] = np.asarray(y_pred).reshape(-1)
    out["residual"] = out["y_true"] - out["y_pred"]
    return out

def _find_ar1_feature(df: pd.DataFrame) -> str:
    candidates = ["ret_lag1","ret_lag_1","ret_l1","lag1_ret","return_lag1","logret_lag1"]
    for c in candidates:
        if c in df.columns:
            return c
    fuzzy = [c for c in df.columns if ("lag" in c.lower() and "ret" in c.lower() and "1" in c)]
    if len(fuzzy)>0:
        return fuzzy[0]
    df["ar1_x"] = df.groupby("Ticker", sort=False)["target_ret"].shift(1)
    return "ar1_x"

all_metrics = []

# 6.1 baseline_zero
model_name = "baseline_zero"
pred_df = _basic_pred_frame(df, np.zeros(len(df)), model_name)
_save_pred_df(pred_df, model_name)
m = _metrics_from_pred(pred_df, model_name)
all_metrics.append(m)
print(m)

# 6.2 baseline_ticker_mean (TRAIN only)
model_name = "baseline_ticker_mean"
ticker_mean = train_df.groupby("Ticker")["target_ret"].mean()
global_mean = float(train_df["target_ret"].mean())
y_pred = df["Ticker"].map(ticker_mean).fillna(global_mean).to_numpy()
pred_df = _basic_pred_frame(df, y_pred, model_name)
_save_pred_df(pred_df, model_name)
m = _metrics_from_pred(pred_df, model_name)
m = _attach_matched_oos_r2(m, pred_df)
all_metrics.append(m)
print(m)

# 6.3 AR1 per ticker
model_name = "ar1_per_ticker"
ar1_col = _find_ar1_feature(df)
print("AR1 regressor column:", ar1_col)

pred_rows = []
for tkr, g in df.sort_values(["Ticker","Date"]).groupby("Ticker", sort=False):
    g = g.dropna(subset=[ar1_col,"target_ret"]).copy()
    if len(g) < 50:
        yhat = np.full(len(g), float(ticker_mean.get(tkr, global_mean)))
        pred_rows.append(_basic_pred_frame(g, yhat, model_name))
        continue

    g_tr = g[g["split"]=="train"]
    if len(g_tr) < 30:
        yhat = np.full(len(g), float(ticker_mean.get(tkr, global_mean)))
        pred_rows.append(_basic_pred_frame(g, yhat, model_name))
        continue

    lr = LinearRegression()
    lr.fit(g_tr[[ar1_col]].to_numpy(), g_tr["target_ret"].to_numpy())
    yhat = lr.predict(g[[ar1_col]].to_numpy())
    pred_rows.append(_basic_pred_frame(g, yhat, model_name))

pred_df = pd.concat(pred_rows, ignore_index=True)
_save_pred_df(pred_df, model_name)
m = _metrics_from_pred(pred_df, model_name)
m = _attach_matched_oos_r2(m, pred_df)
all_metrics.append(m)
print(m)

# 6.4 ridge_pooled
model_name = "ridge_pooled"
num_features = feature_candidates_base.copy()
cat_features = ["Ticker"]

allnan = [c for c in num_features if train_df[c].isna().all()]
if allnan:
    print("Dropping all-NaN train features:", allnan[:20])
    num_features = [c for c in num_features if c not in allnan]

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("impute", SimpleImputer(strategy="median")),
            ("scale", StandardScaler())
        ]), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ],
    remainder="drop"
)

pipe = Pipeline([
    ("prep", preprocess),
    ("model", Ridge(alpha=1000.0, random_state=42))
])

pipe.fit(train_df[num_features + cat_features], train_df["target_ret"])
yhat = pipe.predict(df[num_features + cat_features])
pred_df = _basic_pred_frame(df, yhat, model_name)
_save_pred_df(pred_df, model_name)
m = _metrics_from_pred(pred_df, model_name)
m = _attach_matched_oos_r2(m, pred_df)
all_metrics.append(m)
print(m)

# 6.5 rf_pooled
model_name = "rf_pooled"
rf_preprocess = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ],
    remainder="drop"
)

rf_pipe = Pipeline([
    ("prep", rf_preprocess),
    ("model", RandomForestRegressor(
        n_estimators=400,
        min_samples_leaf=50,
        random_state=42,
        n_jobs=-1
    ))
])

rf_pipe.fit(train_df[num_features + cat_features], train_df["target_ret"])
yhat = rf_pipe.predict(df[num_features + cat_features])
pred_df = _basic_pred_frame(df, yhat, model_name)
_save_pred_df(pred_df, model_name)
m = _metrics_from_pred(pred_df, model_name)
m = _attach_matched_oos_r2(m, pred_df)
all_metrics.append(m)
print(m)

# Save baseline metrics
baseline_metrics = pd.concat(all_metrics, ignore_index=True)
col_order = ["model","split","n","MAE","RMSE","Corr","DirectionalAcc","OOS_R2_vs_baseline"]
for c in col_order:
    if c not in baseline_metrics.columns:
        baseline_metrics[c] = np.nan
baseline_metrics = baseline_metrics[col_order].sort_values(["split","RMSE"]).reset_index(drop=True)

out_path = TAB_DIR / "baseline_metrics.csv"
baseline_metrics.to_csv(out_path, index=False)
print("Saved baseline metrics:", out_path)
baseline_metrics

Saved predictions: stock_project/reports/predictions/baseline_zero.parquet | rows=19,776
           model  split     n       MAE      RMSE  Corr  DirectionalAcc
0  baseline_zero  train  7720  0.012403  0.019210   NaN        0.461140
1  baseline_zero    val  6048  0.019788  0.029259   NaN        0.477844
2  baseline_zero   test  6008  0.016531  0.024833   NaN        0.457390
Saved predictions: stock_project/reports/predictions/baseline_ticker_mean.parquet | rows=19,776
                  model  split     n       MAE      RMSE      Corr  \
0  baseline_ticker_mean  train  7720  0.012353  0.019176  0.025203   
1  baseline_ticker_mean    val  6048  0.019762  0.029256  0.007244   
2  baseline_ticker_mean   test  6008  0.016471  0.024781  0.015846   

   DirectionalAcc  OOS_R2_vs_baseline  
0        0.538860                 0.0  
1        0.522156                 0.0  
2        0.542610                 0.0  
AR1 regressor column: ret_lag1
Saved predictions: stock_project/reports/predictions/ar

,model,split,n,MAE,RMSE,Corr,DirectionalAcc,OOS_R2_vs_baseline
0,baseline_ticker_mean,test,6008,0.016471,0.024781,0.015846,0.542610,0.000000
1,ar1_per_ticker,test,6008,0.016473,0.024786,0.020092,0.542610,-0.000373
2,baseline_zero,test,6008,0.016531,0.024833,NaN,0.457390,NaN
3,ridge_pooled,test,6008,0.017224,0.025323,0.039794,0.460220,-0.044188
4,rf_pooled,test,6008,0.019257,0.027006,0.045119,0.462051,-0.187616
5,rf_pooled,train,7720,0.011018,0.017436,0.580693,0.704793,0.173290
6,ridge_pooled,train,7720,0.012329,0.019092,0.098605,0.540026,0.008793
7,ar1_per_ticker,train,7720,0.012353,0.019163,0.044651,0.536269,0.001359
8,baseline_ticker_mean,train,7720,0.012353,0.019176,0.025203,0.538860,0.000000
9,baseline_zero,train,7720,0.012403,0.019210,NaN,0.461140,NaN


In [15]:
# ==========================
#  Deep Learning Utilities (GRU)
# ==========================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

tf.random.set_seed(SEED)

def make_sequences(df: pd.DataFrame, feature_cols: list[str], seq_len: int):
    df = df.sort_values(["Ticker","Date"]).copy()
    X_list, y_list, meta_rows = [], [], []

    for tkr, g in df.groupby("Ticker", sort=False):
        g = g.dropna(subset=feature_cols + ["target_ret"]).copy()
        vals = g[feature_cols].to_numpy(dtype=np.float32)
        y = g["target_ret"].to_numpy(dtype=np.float32)

        for i in range(seq_len, len(g)):
            X_list.append(vals[i-seq_len:i, :])
            y_list.append(y[i])
            meta_rows.append({
                "Ticker": tkr,
                "Date": g.iloc[i]["Date"],
                "target_date": g.iloc[i]["target_date"],
                "split": g.iloc[i]["split"],
            })

    if len(X_list) == 0:
        raise ValueError("No sequences created. Check NaNs / feature_cols / seq_len.")

    X = np.stack(X_list, axis=0)
    y = np.array(y_list, dtype=np.float32)
    meta = pd.DataFrame(meta_rows)
    return X, y, meta

def build_gru(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.GRU(64),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")
    return model

print("DL ready: make_sequences, build_gru")

DL ready: make_sequences, build_gru


In [16]:
# ==========================
# GRU Baseline (NO GARCH)
# ==========================
SEQ_LEN = 30
FEATURE_COLS = feature_candidates_base  # consistent

df = panel.copy()
df["Date"] = pd.to_datetime(df["Date"])
df["target_date"] = pd.to_datetime(df["target_date"])
df = df.dropna(subset=["split","target_ret"])

X, y, meta = make_sequences(df, FEATURE_COLS, seq_len=SEQ_LEN)
print("X:", X.shape, "y:", y.shape)

idx_train = meta["split"].values == "train"
idx_val   = meta["split"].values == "val"
idx_test  = meta["split"].values == "test"

X_train, y_train = X[idx_train], y[idx_train]
X_val, y_val     = X[idx_val], y[idx_val]
X_test, y_test   = X[idx_test], y[idx_test]
print("Train/Val/Test:", X_train.shape, X_val.shape, X_test.shape)

# scale using TRAIN only
scaler = StandardScaler()
F = X.shape[-1]
scaler.fit(X_train.reshape(-1, F))

def scale_X(X_in):
    X2 = X_in.reshape(-1, F)
    X2s = scaler.transform(X2)
    return X2s.reshape(X_in.shape)

X_train_s = scale_X(X_train)
X_val_s   = scale_X(X_val)
X_test_s  = scale_X(X_test)

model = build_gru((SEQ_LEN, F))
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)
]

history = model.fit(
    X_train_s, y_train,
    validation_data=(X_val_s, y_val),
    epochs=50,
    batch_size=256,
    verbose=1,
    callbacks=callbacks
)

yhat_train = model.predict(X_train_s, verbose=0).reshape(-1)
yhat_val   = model.predict(X_val_s, verbose=0).reshape(-1)
yhat_test  = model.predict(X_test_s, verbose=0).reshape(-1)

pred_train = meta.loc[idx_train, ["Date","target_date","Ticker","split"]].copy()
pred_val   = meta.loc[idx_val,   ["Date","target_date","Ticker","split"]].copy()
pred_test  = meta.loc[idx_test,  ["Date","target_date","Ticker","split"]].copy()

for p, yt, yp in [(pred_train,y_train,yhat_train),(pred_val,y_val,yhat_val),(pred_test,y_test,yhat_test)]:
    p["model"] = "gru_base"
    p["y_true"] = yt
    p["y_pred"] = yp
pred_df = pd.concat([pred_train,pred_val,pred_test], ignore_index=True)
pred_df["residual"] = pred_df["y_true"] - pred_df["y_pred"]

pred_path = PRED_DIR / "gru_base.parquet"
pred_df.to_parquet(pred_path, index=False)
print("Saved:", pred_path, "rows:", len(pred_df))

# metrics
rows = []
for split in ["train","val","test"]:
    d = pred_df[pred_df["split"]==split]
    m = regression_metrics(d["y_true"].to_numpy(), d["y_pred"].to_numpy())
    rows.append({"model":"gru_base","split":split,"n":int(len(d)),**m})
metrics_df = pd.DataFrame(rows)

oos = oos_r2_vs_baseline_matched(pred_df, PRED_DIR / "baseline_ticker_mean.parquet")
metrics_df["OOS_R2_vs_baseline"] = metrics_df["split"].map(oos)

out_path = TAB_DIR / "gru_base_metrics.csv"
metrics_df.to_csv(out_path, index=False)
print("Saved:", out_path)
metrics_df

X: (19536, 30, 18) y: (19536,)
Train/Val/Test: (7480, 30, 18) (6048, 30, 18) (6008, 30, 18)
Epoch 1/50
30/30 [==============================] - 3s 23ms/step - loss: 0.0176 - val_loss: 0.1799 - lr: 0.0010
Epoch 2/50
30/30 [==============================] - 0s 11ms/step - loss: 0.0028 - val_loss: 0.0823 - lr: 0.0010
Epoch 3/50
30/30 [==============================] - 0s 11ms/step - loss: 0.0017 - val_loss: 0.0647 - lr: 0.0010
Epoch 4/50
30/30 [==============================] - 0s 8ms/step - loss: 0.0014 - val_loss: 0.0516 - lr: 0.0010
Epoch 5/50
30/30 [==============================] - 0s 8ms/step - loss: 0.0011 - val_loss: 0.0471 - lr: 0.0010
Epoch 6/50
30/30 [==============================] - 0s 8ms/step - loss: 9.8979e-04 - val_loss: 0.0402 - lr: 0.0010
Epoch 7/50
30/30 [==============================] - 0s 8ms/step - loss: 8.6931e-04 - val_loss: 0.0403 - lr: 0.0010
Epoch 8/50
30/30 [==============================] - 0s 8ms/step - loss: 7.9207e-04 - val_loss: 0.0333 - lr: 0.0010
Epoch

,model,split,n,MAE,RMSE,Corr,DirectionalAcc,OOS_R2_vs_baseline
0,gru_base,train,7480,0.013821,0.019308,0.379172,0.595856,0.000886
1,gru_base,val,6048,0.138582,0.159862,0.043761,0.520503,-28.857373
2,gru_base,test,6008,0.104815,0.118774,-0.036799,0.541944,-21.972123
